# ## Install Dependencies


In [17]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [18]:
# Verify installations
import numpy as np
import sklearn
import hazm
import json

# Persian Text Retrieval System
# This notebook implements a simple Persian text retrieval system using the `IR_data_news_12k.json` dataset. The system uses TF-IDF for ranking documents and evaluates performance using precision and recall.

# ## 1. Load the Dataset
# Load the `IR_data_news_12k.json` dataset.


In [19]:
def read_json(path):
  file = open(path)
  data = json.load(file)
  return data

In [20]:
input_data = read_json('./data/IR_data_news_12k.json')

In [21]:
print(list(input_data.values())[0])

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا', 'category': 'sports'}


In [22]:
print(list(input_data.values())[0].keys())
contents = [input_data[i]['content'] for i in input_data]
print(len(contents))
print(contents[0])

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])
12202

به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/





# ## 2. Preprocess the Data
# Use the `Hazm` library to normalize, tokenize, and remove stop words from the text.


In [23]:
from parsivar import Normalizer, Tokenizer, FindStems
from hazm import stopwords_list

In [24]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

In [25]:
stopwords = {stopwords_list()[i] for i in range(0, len(stopwords_list()) - 1)}
extra_stopwords = ['،', '.', ')', '(', '}', '{', '«', '»', '؛', ':',  '؟','>','<','|','+','-','*',"^",'%','#','=','_','/','«','»','$','[',']','&',"❊",'«','»']
stopwords.update(extra_stopwords)

In [26]:
def preprocess(contents, rm_sw=True, stemming=True):
  preprocessed_docs = []
  for content in contents:
    
    # normalizing
    normalized_content = normalizer.normalize(content)
    content_tokens = tokenizer.tokenize_words(normalized_content)
    tokens = []
    for token in content_tokens:
      # stemming
      if stemming:
        token = stemmer.convert_to_stem(token)
      # remove stopwords
      if rm_sw:
          if token in stopwords:
                continue
      tokens.append(token)
    preprocessed_docs.append(tokens)
    # tokens of each doc
  return preprocessed_docs

In [27]:
preprocessed_docs = preprocess(contents)

In [28]:
print(len(preprocessed_docs))

12202


# ## 3. Positional Indexing

In [29]:
class Term:
    def __init__(self):
        self.total_freq = 0
        self.pos_in_doc = {} 
        self.freq_in_doc = {}

    def update_posting(self, doc_id, term_position):
      if doc_id not in self.pos_in_doc:
            self.pos_in_doc[doc_id] = []
            self.freq_in_doc[doc_id] = 0 
      self.pos_in_doc[doc_id].append(term_position)
      self.freq_in_doc[doc_id] += 1
      self.total_freq += 1

In [30]:
def positional_indexing(preprocessed_docs):
    p_inv_index = {} 
    for doc_id in range(len(preprocessed_docs)):
        for pos in range(len(preprocessed_docs[doc_id])):
            term = preprocessed_docs[doc_id][pos]
            if term in p_inv_index:
                term_obj = p_inv_index[term]
            else:
                term_obj = Term()
            term_obj.update_posting(doc_id, pos)
            p_inv_index[term] = term_obj

    return p_inv_index

In [31]:
positional_index = positional_indexing(preprocessed_docs)

In [32]:
print(positional_index['بایرن'].pos_in_doc)

{98: [49], 101: [46], 110: [29, 57], 142: [335], 150: [73], 178: [9, 30, 40], 222: [125], 225: [44], 300: [27, 56], 313: [218], 316: [125], 357: [74], 370: [144], 384: [5, 34], 386: [5, 70], 405: [12], 413: [72], 434: [5, 24], 443: [11, 25, 37, 67], 444: [77], 492: [108], 583: [57], 706: [17], 712: [18], 724: [13, 73, 88, 117], 747: [9, 42, 53], 759: [9], 760: [6, 34, 52], 791: [32], 813: [10, 25, 59, 103], 836: [43, 52], 863: [38, 51, 96], 934: [55], 948: [45, 64], 968: [71, 86], 1152: [38], 1156: [29], 1245: [8, 16, 22], 1285: [15, 26, 35, 55, 91, 101], 1303: [33, 205], 1313: [94], 1421: [37], 1636: [24], 1637: [9, 17, 28], 1701: [99], 1716: [64, 75], 1723: [8, 39], 1954: [39], 1996: [29], 2005: [5, 31, 42], 2013: [60, 77, 92], 2064: [10, 35, 46, 62, 68], 2201: [39, 47], 2306: [9, 35, 41], 2363: [10, 31, 55, 78, 84, 99, 133, 143, 178, 184, 189, 198, 205, 211, 218, 223, 226, 248, 268, 275, 311, 324, 336, 351], 2421: [48], 2509: [25], 2516: [74], 2518: [223, 241], 2526: [63], 2629: [22

In [33]:
preprocessed_docs[222]

['گزارش',
 'خبرگزاری',
 'فارس',
 'سایت',
 'weallfollowunited',
 'انگلیس',
 'گزارش',
 'حضور',
 'نماینده',
 'باشگاه',
 'منچستر',
 'یونایتد',
 'بازی',
 'پورتو',
 'لیون',
 'بازی',
 'مرحله',
 'هشتم',
 'نهایی',
 'لیگ',
 'اروپا',
 'خبر',
 'بازی',
 'پورتو',
 'لیون',
 'نتیجه',
 'صفر',
 'نفع',
 'تیم',
 'فرانسوی',
 'پایان',
 'مهدی',
 'طارم',
 'ستاره',
 'تیم',
 'ملی',
 'کشور',
 'بازی',
 'برخلاف',
 'بازی',
 'گذشته',
 'توانست&توان',
 'عمل\u200cکرد',
 'رسانه',
 'انگلیسی',
 'گزارش',
 'عنوان',
 'استعدادیاب\u200cهای',
 'باشگاه',
 'منچستر',
 'ورزشگاه',
 'پورتو',
 'حضور',
 'کرد&کن',
 'ستاره',
 'تیم',
 'زیرنظر',
 'گرفت&گیر',
 'تابستان',
 'اقداماتی',
 'جذب',
 'آن\u200cها',
 'انجام',
 'شد&شو',
 'مهدی',
 'طارم',
 'ستاره',
 'ایرانی',
 'یکی',
 'ستارگانی\u200cبود',
 'رادار',
 'نماینده',
 'منچستر',
 'یونایتد',
 'قرار',
 'گرفت&گیر',
 'توانست&توان',
 'عمل\u200cکرد',
 'درخشانید&درخشان',
 'مقابل',
 'دیده',
 'مسئول',
 'شیاطین',
 'سرخ',
 'رسانه',
 'نوشت&نویس',
 'منچستریونایتد',
 'نماینده',
 'حضور',
 'بازی',
 'پورتو',
 

In [34]:
# check the value of total_freq is correct or not
print(positional_index['مونیخ'].total_freq)
print(sum(positional_index['مونیخ'].freq_in_doc.values()))

208
208


## 4. Answering Query

In [36]:
from itertools import permutations
import re

In [43]:
def process_phrase(tokens):

    result = []
# used when we have more than 2 words in our phrase
# split it to 2 biword index
# aggregate the results
    for biword in permutations(tokens, 2):
        w1 = biword[0]
        w2 = biword[1]
        if (w1 not in positional_index.keys()) or (w2 not in positional_index.keys()):
            return []
        
        indx1 = tokens.index(w1)
        indx2 = tokens.index(w2)
        pos_dic_1 = positional_index.get(w1).pos_in_doc
        pos_dic_2 = positional_index.get(w2).pos_in_doc  
        k = abs(indx1-indx2)

        docs = positional_intersect(pos_dic_1, pos_dic_2, k)
        
        if len(result) == 0:
            result = docs
        else:
            result = list(set(result) & set(docs))

    return result

In [39]:
def process_query(not_words=[], phrases=[], words=[]):
    ranks={}
    
    # find words
    for token in words:
        if token in positional_index.keys():
            for doc_id in positional_index[token].pos_in_doc.keys():
                if doc_id in ranks.keys():
                    ranks[doc_id]+=1
                else:
                    ranks[doc_id]=1
    # find phrases
    for phrase in phrases:
        for doc_id in process_phrase(phrase):
            if doc_id in ranks.keys():
                ranks[doc_id] += 1
            else:
                ranks[doc_id] = 1
    # find ! not words
    not_words_docs = []
    for word in not_words:
        doc_ids = positional_index[word].pos_in_doc.keys()
        for doc_id in doc_ids:
            not_words_docs.append(doc_id)
            
    # from results remove docs which contain not
    if len(ranks) > 0:
        for doc in not_words_docs:
            if doc in ranks.keys():
                del ranks[doc]
                
    ranks = dict(sorted(ranks.items(), key=lambda x: x[1], reverse=True))
    
    return ranks

def not_terms(query):
    splitted_query = query.split()
    indices = [i for i in range(len(splitted_query)) if splitted_query[i]=='!']
    result = [splitted_query[i+1] for i in indices]
    return result

def get_phrase(query):
    res = []
    quoted = re.compile('"[^"]*"')
    for value in quoted.findall(query):
        value = value.replace('"', '').strip().split()
        res.append(value)
    return res

def search_query(query):
    # preprocessed query
    query = ' '.join(preprocess([query], True, True)[0])
    phrases = get_phrase(query)
    flat_phrases = [item for sublist in phrases for item in sublist]
    not_words = not_terms(query)
    query = query.replace('"','')
    query = query.replace('!', '')
    splitted_query = query.split()
    looking_words = []
    for x in splitted_query:
      if x not in not_words and x not in flat_phrases:
        looking_words.append(x)
    output = process_query(not_words=not_words, phrases=phrases, words=looking_words)  
    return output

def print_output(output_dict):
    ids = list(output_dict.keys())[:5]
    for i in range(len(ids)):
        print(f'Rank {i + 1}:')
        title = input_data[str(ids[i])]['title']
        url = input_data[str(ids[i])]['url']
        print('title: ', title, '\nurl: ', url)
        print('------------')

In [40]:
query = 'تحریم‌های آمریکا علیه ایران'
res = search_query(query)
print_output(res)

Rank 1:
title:  خبرگزاری فارس ۱۹ ساله شد 
url:  https://www.farsnews.ir/news/14001122000809/خبرگزاری-فارس-۱۹-ساله-شد
------------
Rank 2:
title:  اصولی: فدراسیون فوتبال جمهوری اسلامی ایران هستیم نه جزیره مستقل/ با گفتار ساختارشکنانه فدراسیون را به ناکجا آباد می‌برند 
url:  https://www.farsnews.ir/news/14001117000518/اصولی-فدراسیون-فوتبال-جمهوری-اسلامی-ایران-هستیم-نه-جزیره-مستقل-با
------------
Rank 3:
title:  احتمال مبادله نازنین زاغری در ازای 530میلیون دلار 
url:  https://www.farsnews.ir/news/14001223001080/احتمال-مبادله-نازنین-زاغری-در-ازای-530میلیون-دلار
------------
Rank 4:
title:  متکی: آمریکا با ابزار ناتو به دنبال تجزیه روسیه است 
url:  https://www.farsnews.ir/news/14001222000749/متکی-آمریکا-با-ابزار-ناتو-به-دنبال-تجزیه-روسیه-است
------------
Rank 5:
title:  توضیحات یک منبع آگاه درباره وقفه مذاکرات وین 
url:  https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
------------


In [41]:
query = 'تحریم‌های آمریکا ! ایران'
res = search_query(query)
print_output(res)

Rank 1:
title:  ادامه تحریم‌های سیاسی علیه المپیک پکن/ژاپن هم به صف منتقدان پیوست 
url:  https://www.farsnews.ir/news/14001003000306/ادامه-تحریم‌های-سیاسی-علیه-المپیک-پکن-ژاپن-هم-به-صف-منتقدان-پیوست
------------
Rank 2:
title:  انتقاد دانشجویان ایرانی در اروپا به برخورد دوگانه مدعیان حقوق بشر با قضایای اوکراین و جنایت‌های آل سعود 
url:  https://www.farsnews.ir/news/14001224000014/انتقاد-دانشجویان-ایرانی-در-اروپا-به-برخورد-دوگانه-مدعیان-حقوق-بشر-با
------------
Rank 3:
title:  محو رژیم صهیونیستی از آرمان‌های نظام اسلامی حذف نشده است 
url:  https://www.farsnews.ir/news/14001222000379/محو-رژیم-صهیونیستی-از-آرمان‌های-نظام-اسلامی-حذف-نشده-است
------------
Rank 4:
title:  تجربه نشان داده به عهد آمریکا در مذاکرات نمی‌شود اعتماد کرد 
url:  https://www.farsnews.ir/news/14001203000366/تجربه-نشان-داده-به-عهد-آمریکا-در-مذاکرات-نمی‌شود-اعتماد-کرد
------------
Rank 5:
title:  سود مافیای اسلحه‌سازی آمریکا در ناامن بودن جهان است 
url:  https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-

In [54]:
query = 'کنگره ضدتروریست'
res = search_query(query)
print_output(res)

Rank 1:
title:  توضیحات یک منبع آگاه درباره وقفه مذاکرات وین 
url:  https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
------------
Rank 2:
title:  بحران دوباره گریبان وزنه‌برداری را گرفت/زیرپا گذاشتن قوانین در IWF 
url:  https://www.farsnews.ir/news/14001223000130/بحران-دوباره-گریبان-وزنه‌برداری-را-گرفت-زیرپا-گذاشتن-قوانین-در-IWF
------------
Rank 3:
title:  برگزاری مراسم روز درختکاری در فدراسیون ووشو/ ملی‌پوشان ۷۲ اصله نهال را غرس کردند 
url:  https://www.farsnews.ir/news/14001215000800/برگزاری-مراسم-روز-درختکاری-در-فدراسیون-ووشو-ملی‌پوشان-۷۲-اصله-نهال-را
------------
Rank 4:
title:  «پهلوانان ماندگار؛ ۵۱۳۵ شهید ورزشکار به نیت هر شهید یک درخت»/ کاشت نمادین درخت در فوتبال 
url:  https://www.farsnews.ir/news/14001215000248/پهلوانان-ماندگار-۵۱۳۵-شهید-ورزشکار-به-نیت-هر-شهید-یک-درخت-کاشت-نمادین
------------
Rank 5:
title:  برگزاری سوپرجام فوتبال کشور به نام شهدای چوار 
url:  https://www.farsnews.ir/news/14001019000298/برگزاری-سوپرجام-فوتبال-کشور-به-نا